In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import sys, os

sys.path.append(os.path.abspath('../..'))

In [3]:
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
from tqdm.notebook import tqdm
import joblib

In [4]:
from hit_finding.constants import *
from hit_finding.metrics import *
from learning_tabular.preprocessing import list_columns
from learning_tabular.constants import CHANNELS, FAMILIES, LABEL_FIELD
from learning_tabular.preprocessing import load_plate_csv, list_columns

# from os import chdir, scandir, makedirs
import matplotlib.pyplot as plt
# from matplotlib.gridspec import GridSpec
# import seaborn as sns
import pandas as pd
import numpy as np


In [42]:
if 'alonshp' in os.path.abspath('..'):
    userid = 'alonshp'
else:
    userid = 'naorko'

if userid == 'alonshp':
    res_dir = os.path.abspath('../../../../../../assafzar-group/g-and-n/')
    # res_dir = os.path.abspath('../../../../../assafzar-group/g-and-n/')
    sys.path.append(res_dir)
else:
    sys.path.append(os.path.abspath('..'))

if userid == 'alonshp':
    res_pth = os.path.join(res_dir, 'tabular_models_results/30000/results/z_scores/pure')
    plates = os.listdir(os.path.join(res_pth, 'err'))
else:
    res_pth = '../../../tabular_models_results/30000/results/z_scores/frac_score'
    t = 4
    # plates = os.listdir(os.path.join(res_pth, str(4)))
    res_path_thresh = os.path.join(res_pth, str(t))
    plates = os.listdir(os.path.join(res_pth, str(t)))


debug=True

if debug:
    files = files[0:20]

# plates.sort()
# p = plates[0]

# raw = False
# cp_df = [load_pure_zscores(plate_csv, raw=raw) for _, plate_csv in files]
# cp_df = pd.concat(cp_df)

# raw = True
# raw_pure_plates = [load_pure_zscores(plate_csv, raw=raw)for _, plate_csv in files]
# raw_pure_plates = pd.concat(raw_pure_plates)

files

[(24277, '24277.csv'),
 (24301, '24301.csv'),
 (24300, '24300.csv'),
 (24296, '24296.csv'),
 (24297, '24297.csv')]

In [43]:
# zscores = extract_scores_from_all(extract_score, threshold=4)
# joined = pd.concat(zscores.values())
# joined.shape

cpd_map = pd.concat([load_pure_zscores(p_name, raw=False) for _, p_name in files])
cpd_raw = pd.concat([load_pure_zscores(p_name, raw=True) for _, p_name in files])
cpd_raw1to1 = pd.concat([load_pure_zscores(p_name, raw=True, inter_channel=False) for _, p_name in files])

cpd_raw.drop(columns=[c for c in cpd_raw.columns if c not in cpd_map.columns], inplace=True)

cpd_map.shape, cpd_raw.shape, cpd_raw1to1.shape


((1840, 414), (1920, 414), (1840, 414))

In [44]:
field = 'ALL_map'

In [45]:
cpd_map_trt = cpd_map.query('Metadata_ASSAY_WELL_ROLE == "treated"')
cpd_raw_trt = cpd_raw.query('Metadata_ASSAY_WELL_ROLE == "treated"')
cpd_raw1to1_trt = cpd_raw1to1.query('Metadata_ASSAY_WELL_ROLE == "treated"')

cpd_map_ctl = cpd_map.query('Metadata_ASSAY_WELL_ROLE == "mock"')
cpd_raw_ctl = cpd_raw.query('Metadata_ASSAY_WELL_ROLE == "mock"')
cpd_raw1to1_ctl = cpd_raw1to1.query('Metadata_ASSAY_WELL_ROLE == "mock"')

In [47]:
# find cells with replicates

# compounds_cnt = cpd_map_trt.query(f'{field} >= 0').index.get_level_values(1).value_counts()
compounds_cnt = cpd_map_trt.index.get_level_values(2).value_counts()
dup_compounds = (compounds_cnt[compounds_cnt > 1]).index
replicates = cpd_map_trt[cpd_map_trt.index.isin(dup_compounds, 2)]
cpds = replicates.index.get_level_values(2).unique()
# dup_compounds
dup_compounds

Index(['BRD-K18250272-003-03-7', 'BRD-K18316707-001-01-9',
       'BRD-A14566392-004-03-5', 'BRD-A14574269-051-01-4',
       'BRD-A15034104-002-03-6', 'BRD-A15070535-323-02-1',
       'BRD-A15415227-003-01-1', 'BRD-A15435692-003-01-5',
       'BRD-A15454510-001-02-8', 'BRD-A15530910-001-01-3',
       ...
       'BRD-K27062708-001-04-1', 'BRD-K26979635-001-01-1',
       'BRD-K26383086-001-02-8', 'BRD-K26954192-001-03-5',
       'BRD-K26807903-001-02-7', 'BRD-K26739552-003-04-8',
       'BRD-K26573499-300-01-3', 'BRD-K26548821-001-01-8',
       'BRD-K26521938-001-04-9', 'BRD-K26429091-051-01-9'],
      dtype='object', length=639)

In [49]:
def get_median_score(cpds_list, df):
    """
    This function calculates the median score for each compound based on its replicates
    """
    
    cpds_median_score = {}
    for cpd in cpds_list:
        cpd_replicates = df[df.index.isin([cpd], 2)].copy()
        cor_mat = cpd_replicates.astype('float64').T.corr(method = 'pearson').values
        if len(cor_mat) == 1:
            median_val = 1
        else:
            median_val = np.median(cor_mat[np.triu_indices(len(cor_mat), k = 1)])
            
        cpds_median_score[cpd] = median_val
        
    return cpds_median_score

In [50]:
cpds_med_score = {}
# for method in ['map', 'raw']:
for method in ['map']:
    # cur = replicates.filter(regex=f'_{method}$')
    cpds_med_score[method] = get_median_score(cpds, replicates)

In [53]:
cpds_score_df = pd.DataFrame(cpds_med_score)
cpds_score_df.shape


(639, 1)

In [52]:
def drop_cpds_with_null(df):
    """
    This function drop compounds with median scores of 1 
    or null values in any of the dose points (1-6)
    """
    cpds_with_null = []
    for cpd in df.index:
        if any(df.loc[cpd] == 1) | any(df.loc[cpd].isnull()):
            cpds_with_null.append(cpd)
    df.drop(cpds_with_null, axis = 0, inplace = True)
    
    return df

In [54]:
drop_cpds_with_null(cpds_score_df)
cpds_score_df.shape

(639, 1)

In [56]:
replicates.xs(cpds[0], level=('Metadata_broad_sample'))

,,,Cells_Granularity_10_AGP,Cells_Granularity_11_AGP,Cells_Granularity_12_AGP,Cells_Granularity_13_AGP,Cells_Granularity_14_AGP,Cells_Granularity_15_AGP,Cells_Granularity_16_AGP,Cells_Granularity_1_AGP,Cells_Granularity_2_AGP,Cells_Granularity_3_AGP,...,Cells_Texture_SumAverage_RNA_5_0,Cells_Texture_SumEntropy_RNA_10_0,Cells_Texture_SumEntropy_RNA_3_0,Cells_Texture_SumEntropy_RNA_5_0,Cells_Texture_SumVariance_RNA_10_0,Cells_Texture_SumVariance_RNA_3_0,Cells_Texture_SumVariance_RNA_5_0,Cells_Texture_Variance_RNA_10_0,Cells_Texture_Variance_RNA_3_0,Cells_Texture_Variance_RNA_5_0
Plate,Metadata_ASSAY_WELL_ROLE,Image_Metadata_Well,,,,,,,,,,,,,,,,,,,,,
24277,treated,a01,-0.799813,0.389818,2.329626,-1.068969,-0.842933,-0.260704,-0.152070,0.493787,0.444343,0.312052,...,-0.267341,0.179075,-0.022004,0.163952,0.729163,0.949218,0.873810,0.773451,1.353148,1.265752
24296,treated,a01,2.783732,-0.348363,0.352360,-0.710988,-0.046886,0.272468,-0.567135,4.100033,4.454704,1.882017,...,0.552101,4.009186,4.386040,4.002060,0.646692,0.755773,0.676818,0.436484,1.390249,1.220419


In [ ]:
def save_to_csv(df, path, file_name, compress=None):
    """saves dataframes to csv"""
    
    if not os.path.exists(path):
        os.mkdir(path)
    
    df.to_csv(os.path.join(path, file_name), index=False, compression=compress)

In [ ]:
save_to_csv(df_cpd_med_score.reset_index().rename({'index':'cpd'}, axis = 1), 
            'cellpainting_lvl4_cpd_replicate_datasets', 'cpd_replicate_median_scores.csv')

In [55]:
def no_of_replicates_per_cpd(df, df_lvl4):
    """This function computes the numbers of replicates for each compound"""
    
    dose_list = list(set(df_lvl4['Metadata_dose_recode'].unique().tolist()))[1:7]
    cpds_no_of_reps = {}
    for cpd in df.index:
        num_of_reps = 0
        df_cpd = df_lvl4[df_lvl4['pert_iname'] == cpd].copy()
        for dose in dose_list:
            df_dose = df_cpd[df_cpd['Metadata_dose_recode'] == dose].copy()
            num_of_reps += df_dose.shape[0]
        cpds_no_of_reps[cpd] = num_of_reps // len(dose_list)
    df['no_of_replicates'] = cpds_no_of_reps.values()
    return df

,map
BRD-A00100033-001-04-8,0.457502
BRD-A00267231-001-01-1,0.548110
BRD-A00327403-103-01-9,0.152875
BRD-A00520476-001-03-3,0.116760
BRD-A00827783-001-04-8,0.056185
...,...
BRD-K40255344-001-05-1,0.510271
BRD-K40530731-001-02-5,0.275320
BRD-K40578143-001-01-8,0.305130
BRD-K40656405-001-01-5,0.350919


In [ ]:
df_cpd_med_score = no_of_replicates_per_cpd(cpds_score_df, df_level4_new)

In [115]:
from collections import defaultdict

all_unique_targets = pd.DataFrame()
cp_target_comparisons = defaultdict(list)
for target in cpds[0:5]:
    # target_set = set(replicates.xs(target, level=('Metadata_broad_sample')))

    target_set = replicates.xs(target, level=('Metadata_broad_sample'))
    
    for compare_target in cpds[0:5]:
        if not target == compare_target:
            
            compare_target_set = replicates.xs(compare_target, level=('Metadata_broad_sample'))

            # if len(target_set.intersection(compare_target_set)) > 0:
            # if target in cp_target_comparisons:

            cp_target_comparisons[target].append(compare_target)
            # else:
                # cp_target_comparisons[target] = [compare_target]

    all_unique_targets = pd.concat([all_unique_targets, target_set])

# all_unique_targets_set = set(all_unique_targets)
print(f"Number of unique targets: {len(all_unique_targets)}")
cp_target_comparisons

Number of unique targets: 10


defaultdict(list,
            {'BRD-K18250272-003-03-7': ['BRD-K18316707-001-01-9',
              'BRD-K18438502-001-02-6',
              'BRD-K18550767-001-02-8',
              'BRD-K18574842-323-03-3'],
             'BRD-K18316707-001-01-9': ['BRD-K18250272-003-03-7',
              'BRD-K18438502-001-02-6',
              'BRD-K18550767-001-02-8',
              'BRD-K18574842-323-03-3'],
             'BRD-K18438502-001-02-6': ['BRD-K18250272-003-03-7',
              'BRD-K18316707-001-01-9',
              'BRD-K18550767-001-02-8',
              'BRD-K18574842-323-03-3'],
             'BRD-K18550767-001-02-8': ['BRD-K18250272-003-03-7',
              'BRD-K18316707-001-01-9',
              'BRD-K18438502-001-02-6',
              'BRD-K18574842-323-03-3'],
             'BRD-K18574842-323-03-3': ['BRD-K18250272-003-03-7',
              'BRD-K18316707-001-01-9',
              'BRD-K18438502-001-02-6',
              'BRD-K18550767-001-02-8']})

In [111]:
cp_target_comparisons['BRD-K18250272-003-03-7']
tuple(cpds[0:2])
pri

NameError: name 'pri' is not defined

In [131]:
replicates.loc[(slice(None), slice(None), cp_target_comparisons['BRD-K18250272-003-03-7']), :].reset_index(level=2, drop=True)
cp_subset_df = (
    replicates
        # .xs(cp_subset, level=('Metadata_broad_sample'))
        .loc[(slice(None), slice(None), cp_target_comparisons['BRD-K18250272-003-03-7']), :]
        .reset_index(drop=True)
        # .loc[:, cp_features]
        # .loc[:,:]
        .transpose()
        .astype(float)
        .corr(method="spearman")
)
# replicates.loc[(slice(None), slice(None), 'BRD-K18250272-003-03-7'), :].reset_index(level=2, drop=True)
# cp_target_comparisons[2]
cp_subset_df

target_median_score = (
    cp_subset_df
        .melt(value_name="pairwise_cor", ignore_index=False)
        .dropna()
        .pairwise_cor
        .median()
)

target_median_score

0.2941495004124298

In [ ]:

# Calculate median pairwise correlations for All doses
target_all_dose_cor_df = []
for target in cp_target_comparisons:
    cp_subset = cp_target_comparisons[target]

    cp_subset_df = (
        replicates
            .xs(cp_subset, level=('Metadata_broad_sample'))
            .reset_index(drop=True)
            # .loc[:, cp_features]
            .loc[:,:]
            .transpose()
            .astype(float)
            .corr(method="spearman")
    )

    np.fill_diagonal(cp_subset_df.values, np.nan)

    n_compounds = cp_subset_df.shape[0]

    target_median_score = (
        cp_subset_df
            .melt(value_name="pairwise_cor", ignore_index=False)
            .dropna()
            .pairwise_cor
            .median()
    )

    target_all_dose_cor_df.append([target, "All", target_median_score, n_compounds])

target_all_dose_cor_df = pd.DataFrame(target_all_dose_cor_df)

print(target_all_dose_cor_df.shape)
target_all_dose_cor_df.head()


/tmp/ipykernel_135481/3782632522.py:8: FutureWarning: Passing lists as key for xs is deprecated and will be removed in a future version. Pass key as a tuple instead.
  replicates


KeyError: ('BRD-K18316707-001-01-9', 'BRD-K18438502-001-02-6', 'BRD-K18550767-001-02-8', 'BRD-K18574842-323-03-3')

In [17]:
replicates.index.get_level_values(1).value_counts().value_counts()

1278    1
Name: Metadata_ASSAY_WELL_ROLE, dtype: int64

In [14]:
def get_median_score(cpds_list, df):
    """
    This function calculates the median score for each compound based on its replicates
    """
    
    cpds_median_score = {}
    for cpd in cpds_list:
        cpd_replicates = df[df.index.isin([cpd], 1)].copy()
        cor_mat = cpd_replicates.astype('float64').T.corr(method = 'pearson').values
        if len(cor_mat) == 1:
            median_val = 1
        else:
            median_val = np.median(cor_mat[np.triu_indices(len(cor_mat), k = 1)])
            
        cpds_median_score[cpd] = median_val
        
    return cpds_median_score